In [2]:
import sqlite3
from sqlite3 import Error
import os.path
from os import path

#from create_db import create_connection, create_table, clean_column_names
#from populate_db import extract_data, insert_records
#import query_db as qd
import config
import build_dataset as build

import importlib
import datetime
import re
import numpy as np
import pandas as pd

This notebook explores how we can subset the active sentences data to just inmates who have served at least 75 percent of their sentence.

In [16]:
importlib.reload(build)
importlib.reload(config)

<module 'config' from '/Users/charmainerunes/git/covid_decarceration/files/config.py'>

In [4]:
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

In [5]:
dataset_3 = pd.read_csv('../data/dataset_main_active3.csv')

//anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3063: DtypeWarning: Columns (21,22,29,30,32) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [6]:
dataset_3.head()

,ID,COMMITMENT_PREFIX,EARLIEST_SENTENCE_EFFECTIVE_DT,MOST_SERIOUS_OFFENSE_CODE,INMATE_COMPUTATION_STATUS_FLAG,END_DATE,PROJ_END_DATE,INMATE_RECORD_STATUS_CODE,INMATE_ADMIN_STATUS_CODE,DATE_OF_LAST_INMATE_MOVEMENT,TYPE_OF_LAST_INMATE_MOVEMENT,CURRENT_COMMITMENT_PREFIX,CONTROL_STATUS,GENDER,RACE,BIRTH_DATE,STATE_BORN,ETHNICITY,CITIZENSHIP,PRIMARY_OFFENSE_CODE,NextPrefix,NextStart,NextOffense,new_col,Time_Diff,Recidivate,INFRACTION_PER_SENT,felon_count,misd_count,Primary offense code_x,Description (if needed)_x,Recidivate_Risk_Level,Needed a check?_x,Recidivate_Risk_Level_Lenient,Recidivate_Risk_Level_Harsh,Primary offense code_y,Description (if needed)_y,Current_Offense_Risk_Level,Needed a check?_y,Current_Offense_Risk_Level_Lenient,Current_Offense_Risk_Level_Harsh
0,4,AA,1983-07-12,SELL SCHEDULE II,EXPIRED,1984-07-11,1984-07-11,INACTIVE,INACTIVE,1984-07-11,TERMINATED PAROLE,NaN,REGULAR POPULATION RPOP,MALE,WHITE,1961-10-15,ALASKA,UNKNOWN,BORN IN U.S.,SELL SCHEDULE II,NONE,NaN,0,0,NaN,0.0,0.0,2.0,0.0,NaN,NaN,0.0,NaN,0.0,0.0,SELL SCHEDULE II,Selling drugs on Schedule II - high potential ...,3.0,YES,2.0,4.0
1,6,AA,1973-01-30,WORTHLESS CHECK,EXPIRED,1973-03-28,0001-01-01,INACTIVE,INACTIVE,1975-08-18,TERMINATED PAROLE,NaN,REGULAR POPULATION RPOP,MALE,WHITE,1951-07-17,NORTH CAROLINA,UNKNOWN,BORN IN U.S.,WORTHLESS CHECK,AB,1973-04-11,WORTHLESS CHECK,1973,0.0,1.0,0.0,0.0,1.0,WORTHLESS CHECK,0,1.0,NO,1.0,1.0,WORTHLESS CHECK,0,1.0,NO,1.0,1.0
2,6,AB,1973-04-11,WORTHLESS CHECK,EXPIRED,1975-08-18,1974-08-10,INACTIVE,INACTIVE,1975-08-18,TERMINATED PAROLE,NaN,REGULAR POPULATION RPOP,MALE,WHITE,1951-07-17,NORTH CAROLINA,UNKNOWN,BORN IN U.S.,WORTHLESS CHECK,NONE,NaN,0,0,NaN,0.0,0.0,0.0,27.0,NaN,NaN,0.0,NaN,0.0,0.0,WORTHLESS CHECK,0,1.0,NO,1.0,1.0
3,8,AA,1990-04-09,DWI DRIVING WHILE IMPAIRED,EXPIRED,1990-05-17,1990-10-09,INACTIVE,INACTIVE,1995-09-14,TERMINATED PAROLE,NaN,REGULAR POPULATION RPOP,MALE,WHITE,1963-12-29,NORTH CAROLINA,UNKNOWN,BORN IN U.S.,DWI DRIVING WHILE IMPAIRED,AB,1993-08-30,HABITUAL IMPAIRED DRIVING,1993,3.0,1.0,0.0,0.0,1.0,HABITUAL IMPAIRED DRIVING,0,3.0,NO,3.0,3.0,DWI DRIVING WHILE IMPAIRED,0,2.0,NO,2.0,2.0
4,8,AB,1993-08-30,HABITUAL IMPAIRED DRIVING,EXPIRED,1994-01-26,1994-02-18,INACTIVE,INACTIVE,1995-09-14,TERMINATED PAROLE,NaN,REGULAR POPULATION RPOP,MALE,WHITE,1963-12-29,NORTH CAROLINA,UNKNOWN,BORN IN U.S.,HABITUAL IMPAIRED DRIVING,BA,1995-01-02,HABITUAL IMPAIRED DRIVING,1995,1.0,1.0,0.0,1.0,0.0,HABITUAL IMPAIRED DRIVING,0,3.0,NO,3.0,3.0,HABITUAL IMPAIRED DRIVING,0,3.0,NO,3.0,3.0


In [17]:
grouping_target = config.grouping_target
print(grouping_target)

train_data, test_data, validate_data, active_sentences, active_almost_cmomplete = build.split_and_process(dataset_3, config, grouping_target)

binary
248
Size of active sentences dataset:  32711
Dataset size:  773724


//anaconda3/lib/python3.7/site-packages/pandas/core/indexing.py:845: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[key] = _infer_fill_value(value)
//anaconda3/lib/python3.7/site-packages/pandas/core/indexing.py:966: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[item] = s
//anaconda3/lib/python3.7/site-packages/pandas/core/frame.py:3997: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org

Total Number of Unique IDs: 418845
Total Number of IDs in Test Data: 83769
Total Number of IDs in Train Data: 268061
Total Number of IDs in Validate Data: 67015
Do the IDs add up? True
Does Test Represent 20% of the data? True
Test Represents X% of the data: 0.2
Does Train+Validate Represent 80% of the data? False
Train+Validate Represents X% of the data: 0.8
Does Validate Represent 20% of the Train+Validate Data?: 0.19999940312048609
Does Train Represent 80% of the Train+Validate Data?: 0.8000005968795139


/Users/charmainerunes/git/covid_decarceration/files/build_dataset.py:685: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  def construct_vars_post_impute(df):
/Users/charmainerunes/git/covid_decarceration/files/build_dataset.py:685: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  def construct_vars_post_impute(df):
/Users/charmainerunes/git/covid_decarceration/files/build_dataset.py:685: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  def con

Index(['CONTROL_STATUS', 'OFFENSE_CLEAN', 'Current_Offense_Risk_Level',
       'year_month', 'INFRACTION_PER_SENT', 'misd_count', 'felon_count',
       'sentence_count', 'age_at_sentence', 'unemp_rate', 'ID',
       'COMMITMENT_PREFIX', 'BIRTH_DATE', 'EARLIEST_SENTENCE_EFFECTIVE_DT',
       'END_DATE', 'INMATE_ADMIN_STATUS_CODE', 'NextPrefix',
       'Recidivate_Risk_Level', 'age_cat', 'age_first_offense',
       'juv_first_offense', 'current_crime_violent'],
      dtype='object')


In [18]:
active_almost_cmomplete.head()

,Recidivate_Risk_Level,INFRACTION_PER_SENT,misd_count,felon_count,sentence_count,age_at_sentence,unemp_rate,age_first_offense,juv_first_offense,current_crime_violent,CONTROL_STATUS_HIGH SECURITY/MAX CONTROL HCON,CONTROL_STATUS_INTENSIVE CONTROL ICON,CONTROL_STATUS_MAXIMUM CONTROL MCON,CONTROL_STATUS_MODIFIED HOUSING MODH,CONTROL_STATUS_PROTECTIVE CONTROL PCON,CONTROL_STATUS_REGULAR POPULATION RPOP,CONTROL_STATUS_REST.HOUS. ADMINISTRATIVE RHAP,CONTROL_STATUS_REST.HOUS. DISCIPLINARY RHDP,CONTROL_STATUS_RESTRCT HOUSNG 4 CTL PURP RHCP,OFFENSE_CLEAN_ARMED ROBBERY,OFFENSE_CLEAN_ASSAULT BY STRANGULATION,OFFENSE_CLEAN_ASSAULT INFLICT SERI BODY INJ,OFFENSE_CLEAN_ASSAULT ISI,OFFENSE_CLEAN_ASSAULT ON FEMALE,OFFENSE_CLEAN_ASSAULT ON OFFICER/ST EMPLOYEE,OFFENSE_CLEAN_ASSAULT ON POLICEMAN,OFFENSE_CLEAN_ASSAULT ON PUBLIC OFFICIAL,OFFENSE_CLEAN_AWDW,OFFENSE_CLEAN_AWDW GOV OFFICERS/EMPLOYEES,OFFENSE_CLEAN_AWDWISI,OFFENSE_CLEAN_AWDWWITKISI,OFFENSE_CLEAN_B & E & L,OFFENSE_CLEAN_B & E VEHICLES,OFFENSE_CLEAN_BURGLARY 1ST DEGREE,OFFENSE_CLEAN_BURGLARY 2ND DEGREE,OFFENSE_CLEAN_CHEAT - PROPERTY/SERVICES,OFFENSE_CLEAN_COMMON LAW ROBBERY,OFFENSE_CLEAN_CREDIT CARD THEFT,OFFENSE_CLEAN_DAMAGE TO PROPERTY,OFFENSE_CLEAN_DEL/SELL SCHEDULE II,OFFENSE_CLEAN_DISCHG FIREARM-OCC PROPERTY,OFFENSE_CLEAN_DRIV LICENSE REVOKED,OFFENSE_CLEAN_DWI LEVEL 1,OFFENSE_CLEAN_DWI LEVEL 1 AGGRAVATED,OFFENSE_CLEAN_DWI LEVEL 2,OFFENSE_CLEAN_DWI LEVEL 3,OFFENSE_CLEAN_DWI LEVEL 5,OFFENSE_CLEAN_EMBEZZLEMENT,OFFENSE_CLEAN_FAIL TO REGISTER (SEX OFFENDER,OFFENSE_CLEAN_FELONY B&E,OFFENSE_CLEAN_FORGERY,OFFENSE_CLEAN_HABITUAL FELON,OFFENSE_CLEAN_HABITUAL IMPAIRED DRIVING,OFFENSE_CLEAN_HABITUAL MISDEMEANOR ASSAULT,OFFENSE_CLEAN_IDENTITY FRAUD/THEFT,OFFENSE_CLEAN_INDECENT LIBERTY W/CHILD,OFFENSE_CLEAN_INVOLUNTARY MANSLAUGHTER,OFFENSE_CLEAN_KIDNAPPING 2ND DEGREE,OFFENSE_CLEAN_LARCENY,OFFENSE_CLEAN_LARCENY (OVER $200),OFFENSE_CLEAN_LARCENY AFTER B & E,OFFENSE_CLEAN_LARCENY FROM PERSON,OFFENSE_CLEAN_LARCENY OF FIREARMS,OFFENSE_CLEAN_LARCENY OF MOTOR VEHICLE,OFFENSE_CLEAN_LARCENY OVER $1000,OFFENSE_CLEAN_MAINT ANY PLACE CONTR SUB,OFFENSE_CLEAN_MANSLAUGHTER,OFFENSE_CLEAN_MISD B&E,OFFENSE_CLEAN_MURDER FIRST DEGREE,OFFENSE_CLEAN_MURDER SECOND DEGREE,OFFENSE_CLEAN_OBT PROP BY FALSE PR/CHTS/SER,OFFENSE_CLEAN_OTHER,OFFENSE_CLEAN_POSSESS SCHEDULE I,OFFENSE_CLEAN_POSSESS SCHEDULE II,OFFENSE_CLEAN_POSSESS SCHEDULE VI,OFFENSE_CLEAN_POSSESS WITS SCHEDULE I,OFFENSE_CLEAN_POSSESS WITS SCHEDULE II,OFFENSE_CLEAN_POSSESS WITS SCHEDULE VI,OFFENSE_CLEAN_POSSESSING STOLEN GOODS,OFFENSE_CLEAN_POSSESSION OF FIREARM BY FELON,OFFENSE_CLEAN_RAPE FIRST DEGREE,OFFENSE_CLEAN_RAPE SECOND DEGREE,OFFENSE_CLEAN_RECEIVING STOLEN GOODS,OFFENSE_CLEAN_RECEIVING STOLEN VEHICLE,OFFENSE_CLEAN_ROBBERY W/DANGEROUS WEAPON,OFFENSE_CLEAN_SELL SCHEDULE I,OFFENSE_CLEAN_SELL SCHEDULE II,OFFENSE_CLEAN_SELL SCHEDULE VI,OFFENSE_CLEAN_SEXUAL OFFENSE 2ND DEGREE,OFFENSE_CLEAN_SHOPLIFTING,OFFENSE_CLEAN_SPEED ELUDE ARREST/ATTEMPT/SEC,OFFENSE_CLEAN_SPEEDING ELUDE ARREST OR/ATTEM,OFFENSE_CLEAN_TRAFFICKING SCHEDULE I,OFFENSE_CLEAN_TRAFFICKING SCHEDULE II,OFFENSE_CLEAN_UTTERING FORGEDPAPER/INST/END,OFFENSE_CLEAN_VIOLATION PROTECTIVE ORDER,Current_Offense_Risk_Level_1.0,Current_Offense_Risk_Level_2.0,Current_Offense_Risk_Level_3.0,Current_Offense_Risk_Level_4.0,Current_Offense_Risk_Level_5.0,year_month_1976-01,year_month_1976-03,year_month_1976-04,year_month_1976-05,year_month_1976-06,year_month_1976-07,year_month_1976-08,year_month_1976-09,year_month_1976-10,year_month_1976-11,year_month_1976-12,year_month_1977-01,year_month_1977-02,year_month_1977-03,year_month_1977-06,year_month_1977-07,year_month_1977-08,year_month_1977-09,year_month_1977-10,year_month_1977-11,year_month_1978-01,year_month_1978-02,year_month_1978-03,year_month_1978-04,year_month_1978-05,year_month_1978-06,year_month_1978-07,year_month_1978-08,year_month_1978-10,year_month_1978-11,year_month_1978-12,year_month_1979-01,year_month_1979-02,year_month_1979-03,year_month_1979-04,year_mont

In [19]:
active_almost_cmomplete.columns

Index(['Recidivate_Risk_Level', 'INFRACTION_PER_SENT', 'misd_count',
       'felon_count', 'sentence_count', 'age_at_sentence', 'unemp_rate',
       'age_first_offense', 'juv_first_offense', 'current_crime_violent',
       ...
       'OFFENSE_CLEAN_COMMUNICATING THREATS',
       'OFFENSE_CLEAN_DRUG PARA - USE/POSSESS', 'year_month_1977-12',
       'OFFENSE_CLEAN_UNAUTH USE MOTOR CONVEYANCE', 'year_month_1982-08',
       'CONTROL_STATUS_OUT OF CELL RESTRICTION   OUTC', 'year_month_1976-02',
       'sent_total', 'sent_complete', 'almost_complete'],
      dtype='object', length=662)

In [20]:
active_almost_cmomplete.shape

(10762, 662)

### IGNORE EVERYTHING BELOW - already in the build_dataset.py file

In [21]:
# Create a variable that stores today's date
today = pd.to_datetime('today')
print(today)

2020-06-08 11:01:27.057322


In [24]:
# Check that EARLIEST_SENTENCE_EFFECTIVE_DT, END_DATE is datetime

#active_sentences['EARLIEST_SENTENCE_EFFECTIVE_DT'].dtype
active_sentences['END_DATE'].dtype

dtype('O')

In [25]:
# Convert END_DATE to datetime
active_sentences['END_DATE'] = pd.to_datetime(active_sentences['END_DATE'], yearfirst=True)
active_sentences['END_DATE'].dtype

dtype('<M8[ns]')

In [28]:
active_sentences['sent_total'] = (active_sentences['END_DATE'] - active_sentences['EARLIEST_SENTENCE_EFFECTIVE_DT']).dt.days
active_sentences['sent_complete'] = (today - active_sentences['EARLIEST_SENTENCE_EFFECTIVE_DT']).dt.days
active_sentences['pct_sent_complete'] = active_sentences['sent_complete'] / active_sentences['sent_total']

In [30]:
sentence_subset = active_sentences[['EARLIEST_SENTENCE_EFFECTIVE_DT', 'END_DATE', 'sent_total', 'sent_complete', 'pct_sent_complete']]
sentence_subset.head()

,EARLIEST_SENTENCE_EFFECTIVE_DT,END_DATE,sent_total,sent_complete,pct_sent_complete
0,1985-11-04,2261-01-02,100501,12635,0.125720
1,1996-08-14,2261-01-02,96565,8699,0.090084
2,2015-11-03,2022-02-10,2291,1679,0.732868
3,2020-01-08,2021-07-03,542,152,0.280443
4,1982-11-08,2261-01-02,101593,13727,0.135118


In [31]:
potential_release = sentence_subset.loc[sentence_subset['pct_sent_complete'] >= 0.75, :]
potential_release.shape

(10762, 5)

In [32]:
10762 / 32711

0.3290024762312372

In [14]:
test_subset = build.get_sentence_almost_complete(active_sentences)

KeyError: 'END_DATE'